In [4]:
import xarray as xr
import pandas as pd
import numpy as np
import glob
import os

path_data = "C:/Users/margo/OneDrive/Documents/M1 SSD/Projet_Tutore/Analyse_Meteo/data/raw-yearly-combined/era5_fr_t2m/*.nc"
path_mask = "C:/Users/margo/OneDrive/Documents/M1 SSD/Projet_Tutore/Analyse_Meteo/src/mask_france_boolean.nc"
output_filename = "donnees_carte_35ans_journalier.nc"

print("Démarrage de la création du NetCDF journalier...")

try:
    fichiers = sorted(glob.glob(path_data))
    fichiers_select = fichiers[:420]
    print(f"Fichiers sélectionnés : {len(fichiers_select)}")

    mask = xr.open_dataset(path_mask)
    datasets_journaliers = []

    for i, fichier in enumerate(fichiers_select):
        print(f"Traitement fichier {i+1}/{len(fichiers_select)}...", end=" ")

        try:
            ds = xr.open_dataset(fichier, engine='netcdf4')

            if 'valid_time' in ds.coords:
                ds = ds.rename({'valid_time': 'time'})

            ds_masked = ds.where(mask['mask'] == 1)
            ds_masked['t2m'] = ds_masked['t2m'] - 273.15

            ds_daily = ds_masked.resample(time='1D').mean(skipna=True)

            if 't2m' in ds_daily:
                ds_daily['t2m'] = ds_daily['t2m'].astype('float32')

            datasets_journaliers.append(ds_daily)
            ds.close()
            print("Ok")

        except Exception as e_file:
            print(f"Erreur sur {fichier} : {e_file}")

    ds_final = xr.concat(datasets_journaliers, dim='time')
    ds_final = ds_final[['t2m']]
    ds_final = ds_final.rename({'t2m': 'Temperature_C'})

    encoding = {'Temperature_C': {'zlib': True, 'complevel': 5}}
    ds_final.to_netcdf(output_filename, encoding=encoding)

    print("Terminé. Fichier créé.")

except Exception as e:
    print(f"Erreur : {e}")

Démarrage de la création du NetCDF journalier...
Fichiers sélectionnés : 420
Traitement fichier 1/420... Ok
Traitement fichier 2/420... Ok
Traitement fichier 3/420... Ok
Traitement fichier 4/420... Ok
Traitement fichier 5/420... Ok
Traitement fichier 6/420... Ok
Traitement fichier 7/420... Ok
Traitement fichier 8/420... Ok
Traitement fichier 9/420... Ok
Traitement fichier 10/420... Ok
Traitement fichier 11/420... Ok
Traitement fichier 12/420... Ok
Traitement fichier 13/420... Ok
Traitement fichier 14/420... Ok
Traitement fichier 15/420... Ok
Traitement fichier 16/420... Ok
Traitement fichier 17/420... Ok
Traitement fichier 18/420... Ok
Traitement fichier 19/420... Ok
Traitement fichier 20/420... Ok
Traitement fichier 21/420... Ok
Traitement fichier 22/420... Ok
Traitement fichier 23/420... Ok
Traitement fichier 24/420... Ok
Traitement fichier 25/420... Ok
Traitement fichier 26/420... Ok
Traitement fichier 27/420... Ok
Traitement fichier 28/420... Ok
Traitement fichier 29/420... Ok
Trai

In [ ]:
import xarray as xr
import glob
import os
import numpy as np

# ==============================================================================
# 1. CONFIGURATION DES CHEMINS
# ==============================================================================
# Chemin vers tes données brutes (L'autre dossier)
# Le r"..." est important pour Windows
PATH_DATA = r"C:/Users/margo/OneDrive/Documents/M1 SSD/Projet_Tutore/Analyse_Meteo/data/raw-yearly-combined/era5_fr_t2m/*.nc"

# Nom du fichier final qui sera créé dans le dossier actuel
OUTPUT_FILENAME = "donnees_carte_70ans_journalier.nc"

# DÉFINITION DU RECTANGLE FRANCE (Au lieu du masque)
# On prend large pour tout inclure
LAT_MAX = 52.0  # Nord (Lille est à ~50.6)
LAT_MIN = 41.0  # Sud (Corse comprise, Bonifacio ~41.3)
LON_MIN = -6.0  # Ouest (Brest ~-4.5)
LON_MAX = 10.0  # Est (Strasbourg ~7.7, Corse ~9.5)

print("🚀 Démarrage du traitement SANS masque externe...")
print(f"📂 Source : {PATH_DATA}")
print(f"✂️  Découpe : Lat [{LAT_MIN}, {LAT_MAX}] / Lon [{LON_MIN}, {LON_MAX}]")

# ==============================================================================
# 2. LISTE DES FICHIERS
# ==============================================================================
fichiers = sorted(glob.glob(PATH_DATA))

# On garde les 70 premières années (840 mois)
# Si tu as des fichiers annuels, adapte ce chiffre (ex: 70 fichiers)
# Si ce sont des fichiers mensuels : 70 * 12 = 840
fichiers_select = fichiers[:840]

if len(fichiers_select) == 0:
    print("❌ ERREUR : Aucun fichier trouvé. Vérifie le chemin PATH_DATA.")
    exit()

print(f"✅ {len(fichiers_select)} fichiers identifiés à traiter.")

# ==============================================================================
# 3. BOUCLE DE TRAITEMENT
# ==============================================================================
datasets_journaliers = []

try:
    for i, fichier in enumerate(fichiers_select):
        # Affichage de la progression sur la même ligne
        print(f"   Traitement fichier {i+1}/{len(fichiers_select)}...", end="\r")

        try:
            with xr.open_dataset(fichier, engine='netcdf4') as ds:

                # 1. Harmonisation du nom de la variable temps
                if 'valid_time' in ds.coords:
                    ds = ds.rename({'valid_time': 'time'})

                # 2. DÉCOUPE GÉOGRAPHIQUE (CROP)
                # Astuce : ERA5 trie souvent les latitudes de 90 à -90 (décroissant).
                # .sel(slice()) attend souvent des valeurs croissantes.
                # On trie d'abord pour éviter une sélection vide.
                ds = ds.sortby('latitude')
                ds = ds.sortby('longitude')

                # Sélection du rectangle (Slice)
                ds_cropped = ds.sel(
                    latitude=slice(LAT_MIN, LAT_MAX),
                    longitude=slice(LON_MIN, LON_MAX)
                )

                # 3. CONVERSION D'UNITÉS (Kelvin -> Celsius)
                # On suppose que la variable s'appelle 't2m'. Adapte si besoin.
                if 't2m' in ds_cropped:
                    ds_cropped['t2m'] = ds_cropped['t2m'] - 273.15

                # 4. AGRÉGATION JOURNALIÈRE
                # On passe de données horaires (24 points/jour) à une moyenne par jour
                ds_daily = ds_cropped.resample(time='1D').mean(skipna=True)

                # 5. OPTIMISATION MÉMOIRE (Float32)
                # Réduit la taille du fichier final par 2
                if 't2m' in ds_daily:
                    ds_daily['t2m'] = ds_daily['t2m'].astype('float32')

                datasets_journaliers.append(ds_daily)

        except Exception as e_file:
            print(f"\n⚠️ Erreur sur {fichier} : {e_file}")

    # ==============================================================================
    # 4. FUSION ET SAUVEGARDE
    # ==============================================================================
    print("\n📦 Fusion des données (Concaténation)...")
    ds_final = xr.concat(datasets_journaliers, dim='time')

    # Renommage final pour être propre
    ds_final = ds_final.rename({'t2m': 'Temperature_C'})

    # On ne garde que la variable d'intérêt pour alléger
    ds_final = ds_final[['Temperature_C']]

    print(f"💾 Sauvegarde dans {OUTPUT_FILENAME} (Compression activée)...")

    # Compression zlib niveau 5 pour gagner de la place
    encoding = {'Temperature_C': {'zlib': True, 'complevel': 5}}
    ds_final.to_netcdf(OUTPUT_FILENAME, encoding=encoding)

    print(f"🎉 TERMINÉ ! Fichier généré : {os.path.abspath(OUTPUT_FILENAME)}")

except Exception as e:
    print(f"\n❌ Erreur Générale : {e}")

🚀 Démarrage du traitement SANS masque externe...
📂 Source : C:/Users/margo/OneDrive/Documents/M1 SSD/Projet_Tutore/Analyse_Meteo/data/raw-yearly-combined/era5_fr_t2m/*.nc
✂️  Découpe : Lat [41.0, 52.0] / Lon [-6.0, 10.0]
✅ 840 fichiers identifiés à traiter.
   Traitement fichier 840/840...
📦 Fusion des données (Concaténation)...
💾 Sauvegarde dans donnees_carte_70ans_journalier.nc (Compression activée)...
🎉 TERMINÉ ! Fichier généré : c:\Users\margo\OneDrive\Documents\M1 SSD\Data Visualisation\Projet-Data-Visu\Projet_Data_Visu_Margot_Hanaa\TP\donnees_carte_70ans_journalier.nc


In [ ]:
import xarray as xr
import glob
import os
import pandas as pd
import sys
from pathlib import Path

# ==============================================================================
# 1. CONFIGURATION ET RECHERCHE AUTOMATIQUE
# ==============================================================================
print("🚀 Démarrage de la mise à jour (Extension 2019-2025)...")

# 1. Chemin vers tes données BRUTES (ERA5)
PATH_RAW_DATA = r"C:/Users/margo/OneDrive/Documents/M1 SSD/Projet_Tutore/Analyse_Meteo/data/raw-yearly-combined/era5_fr_t2m/*.nc"

# 2. Nom du NOUVEAU fichier de sortie
OUTPUT_FILENAME = "meteo_france_1950_2025.nc"

# 3. RECHERCHE INTELLIGENTE DE L'ANCIEN FICHIER
# On cherche dans le dossier courant, le dossier parent, et le dossier TP voisin
base_dir = Path(".")
noms_possibles = ["meteo_france_70ans_final.nc"]
chemins_a_tester = [
    base_dir,                       # Dossier TP_Code
    base_dir.parent / "TP",         # Dossier TP (frère)
    base_dir / ".." / "TP",         # Alternative frère
    base_dir / "TP"                 # Sous-dossier éventuel
]

PATH_ANCIEN_FICHIER = None

for dossier in chemins_a_tester:
    for nom in noms_possibles:
        p = dossier / nom
        if p.exists():
            PATH_ANCIEN_FICHIER = p
            break
    if PATH_ANCIEN_FICHIER:
        break

if not PATH_ANCIEN_FICHIER:
    sys.exit("❌ ERREUR : Impossible de trouver 'meteo_france_70ans_final.nc' ni dans TP_Code, ni dans TP.")

print(f"📖 Fichier historique trouvé : {PATH_ANCIEN_FICHIER.resolve()}")

# Délimitation France
LAT_MAX, LAT_MIN = 52.0, 41.0
LON_MIN, LON_MAX = -6.0, 10.0

# ==============================================================================
# 2. CHARGEMENT DE L'HISTORIQUE
# ==============================================================================
try:
    ds_ancien = xr.open_dataset(PATH_ANCIEN_FICHIER)
    var_name_ancien = list(ds_ancien.data_vars)[0]
    print(f"   Variable détectée : {var_name_ancien}")
except Exception as e:
    sys.exit(f"❌ Erreur lecture ancien fichier : {e}")

# ==============================================================================
# 3. TRAITEMENT DES NOUVEAUX FICHIERS (2019-2025)
# ==============================================================================
print("🔍 Recherche des fichiers ERA5 récents...")
tous_fichiers = glob.glob(PATH_RAW_DATA)

annees_a_ajouter = ['2019', '2020', '2021', '2022', '2023', '2024', '2025']
fichiers_new = sorted([f for f in tous_fichiers if any(an in f for an in annees_a_ajouter)])

if not fichiers_new:
    sys.exit("❌ ERREUR : Aucun fichier RAW trouvé (vérifiez le chemin PATH_RAW_DATA).")

print(f"✅ {len(fichiers_new)} fichiers à traiter.")

datasets_new = []

for i, fichier in enumerate(fichiers_new):
    print(f"   ⚡ Traitement {os.path.basename(fichier)} ({i+1}/{len(fichiers_new)})...", end="\r")

    try:
        with xr.open_dataset(fichier, engine='netcdf4') as ds:
            if 'valid_time' in ds.coords: ds = ds.rename({'valid_time': 'time'})

            # Crop
            if 'latitude' in ds.coords: ds = ds.sortby('latitude')
            if 'longitude' in ds.coords: ds = ds.sortby('longitude')
            ds_cropped = ds.sel(latitude=slice(LAT_MIN, LAT_MAX), longitude=slice(LON_MIN, LON_MAX))

            # Conversion
            raw_var = 't2m' if 't2m' in ds_cropped else list(ds_cropped.data_vars)[0]
            ds_daily = ds_cropped.resample(time='1D').mean(skipna=True)

            ds_daily[raw_var] = ds_daily[raw_var] - 273.15
            ds_daily[raw_var] = ds_daily[raw_var].astype('float32')

            # Renommage
            ds_daily = ds_daily.rename({raw_var: var_name_ancien})
            datasets_new.append(ds_daily)

    except Exception as e:
        print(f"\n⚠️ Erreur sur {fichier} : {e}")

# ==============================================================================
# 4. FUSION ET SAUVEGARDE
# ==============================================================================
print("\n\n🔗 Fusion finale...")

if datasets_new:
    ds_nouveau_bloc = xr.concat(datasets_new, dim='time')
    ds_final = xr.concat([ds_ancien, ds_nouveau_bloc], dim='time')

    # Nettoyage doublons
    ds_final = ds_final.sortby('time')
    ds_final = ds_final.drop_duplicates(dim='time')

    start = pd.to_datetime(ds_final.time.values[0]).strftime('%Y-%m-%d')
    end = pd.to_datetime(ds_final.time.values[-1]).strftime('%Y-%m-%d')
    print(f"📅 Période finale : {start} -> {end}")

    # Sauvegarde
    path_sortie = base_dir / OUTPUT_FILENAME
    print(f"💾 Sauvegarde dans : {path_sortie.resolve()}")

    encoding = {var_name_ancien: {'zlib': True, 'complevel': 5}}
    ds_final.to_netcdf(path_sortie, encoding=encoding)

    print("🎉 TERMINÉ ! N'oubliez pas de mettre ce nouveau fichier dans votre dossier Dash.")

else:
    print("❌ Echec : Aucun nouveau dataset valide.")

🚀 Démarrage de la mise à jour (Extension 2019-2025)...
📖 Fichier historique trouvé : C:\Users\margo\OneDrive\Documents\M1 SSD\Data Visualisation\Projet-Data-Visu\Projet_Data_Visu_Margot_Hanaa\TP\meteo_france_70ans_final.nc
   Variable détectée : Temperature_C
🔍 Recherche des fichiers ERA5 récents...
✅ 84 fichiers à traiter.
   ⚡ Traitement era5_t2m_fr_2025_12.nc (84/84)...

🔗 Fusion finale...
📅 Période finale : 1950-01-01 -> 2025-12-11
💾 Sauvegarde dans : C:\Users\margo\OneDrive\Documents\M1 SSD\Data Visualisation\Projet-Data-Visu\Projet_Data_Visu_Margot_Hanaa\TP_Code\meteo_france_1950_2025.nc
🎉 TERMINÉ ! N'oubliez pas de mettre ce nouveau fichier dans votre dossier Dash.
